# **Ejercicios de pair programming Módulo 2 Sprint 2**
## **ETL: Lección 3**
### **Transformación: Clases y funciones de limpieza**

---

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [2]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np

In [3]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [4]:
df_censo_ccaa = pd.read_csv("data/poblacion_comunidades.csv", index_col = 0)

In [5]:
class Extraccion:
    
    def __innit__(self, año_inicio, año_final):
        
        self.año_inicio
        self.año_final
        
    def nacional(self, año_inicio, año_final):  
        df_energia_nacional = pd.DataFrame()
        for año in range(self.año_inicio, self.año_final):

            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year"
            response = requests.get(url=url)

            fichero_json_renovable = response.json()["included"][0]["attributes"]["values"] 
            fichero_json_no_renovable = response.json()["included"][1]["attributes"]["values"]

            df_renovables = pd.json_normalize(fichero_json_renovable)
            df_no_renovables = pd.json_normalize(fichero_json_no_renovable)

            df_renovables["tipo"] = response.json()["included"][0]["type"]
            df_no_renovables["tipo"] = response.json()["included"][1]["type"]
            
            df_energia_nacional = pd.concat([df_energia_nacional, df_renovables, df_no_renovables], axis = 0)
            
        return df_energia_nacional
    
    
    def df_completo_ccaa(self, año_inicio, año_final, cod_comunidades):  
        
        self.cod_comunidades = cod_comunidades
        
        df_completo_ccaa = pd.DataFrame()
        for año in range(self.año_inicio, self.año_final):

            for k, v in cod_comunidades.items():
                
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
            
                response2 = requests.get(url=url)

                fichero_json_no_renovable = response2.json()["included"][0]["attributes"]["values"] 
                
                df_no_renovables = pd.json_normalize(fichero_json_no_renovable)
            
                df_no_renovables["tipo"] = response2.json()["included"][0]["type"]

                df_no_renovables["comunidad"] = k

                df_no_renovables["id_comunidad"] = v

            
                df_completo_ccaa = pd.concat([df_completo_ccaa, df_no_renovables], axis = 0)
     
        return df_completo_ccaa
        
    def limpieza_censo(self, df):
        
        
        df.drop(["Comunidades_y_Ciudades_Autónomas"], axis =1, inplace = True)
        df["Año"]= df["Año"].apply(pd.to_datetime)
        
        return df
        
    
    def limpieza_energias(self, df_energias):
        
        df_energias["value"]=df_energias["value"].round(2)
        df_energias["percentage"]=df_energias["percentage"].round(2)
        df_energias[["fecha","hora"]]=df_energias.datetime.str.split("T",expand = True)
        df_energias.drop(["hora"], axis = 1, inplace=True)
        df_energias["fecha"]=df_energias["fecha"].apply(pd.to_datetime)
        
        return df_energias
       
        
        